# Metodos directos - Basados en gradiente
Para optimización de funciones matematicas suaves (diferenciables al menos 2 veces) sin restricciones. Basado en los notebooks de Kaan Öztürk disponibles en https://github.com/mkozturk/notebooks/tree/master.

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np

Consideremos la función Rosenbrock
$$f(x,y)=10(y-x^2)^2 + (1-x)^2$$
con gradiente
$$\nabla f = \left[\begin{array}{c}
40x^3 - 40xy +2x - 2 \\\
20(y-x^2)
\end{array}\right]$$
y Hessiana
$$\nabla^2 f = \left[
\begin{array}{c}
120x^2-40y+2 & -40x \\\
-40x & 20
\end{array}\right]$$
El unico minimo se encuentra en $(x,y)=(1,1)$ donde $f(1,1)=0$. Construimos las funciones, funcion objetivo, gradiente, Hessiana.

In [ ]:
def objfun(x,y):
    return 10*(y-x**2)**2 + (1-x)**2
def gradient(x,y):
    return np.array([-40*x*y + 40*x**3 -2 + 2*x, 20*(y-x**2)])
def hessian(x,y):
    return np.array([[120*x*x - 40*y+2, -40*x],[-40*x, 20]])

Creamos una funcion en Python que grafica el contorno de la funcion Rosembrock.

In [ ]:
def contourplot(objfun, xmin, xmax, ymin, ymax, ncontours=50, fill=True):

    x = np.linspace(xmin, xmax, 200)
    y = np.linspace(ymin, ymax, 200)
    X, Y = np.meshgrid(x,y)
    Z = objfun(X,Y)
    if fill: # grafica contorno
        plt.contourf(X,Y,Z,ncontours); 
    else:
        plt.contour(X,Y,Z,ncontours);
    plt.scatter(1,1,marker="x",s=50,color="r");  # marcar el minimo

Aqui graficamos una figura del contorno de la funcion Rosembrock, con el minimo global marcado con una X roja.

In [ ]:
contourplot(objfun, -7,7, -10, 40, fill=False)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Contours of $f(x,y)=10(y-x^2)^2 + (1-x)^2$");

# Steepest descent (gradiente descendente) con paso fijo

Primero, escribimos una funcion que utiliza el metodo de Steepest descent. Empieza la solucion en un posicion inicial (`init`), se mueve a traves del opuesto al gradiente con pasos `steplength`, hasta que la diferencia del error absoluto entre valores de la funcion caen por debajo de la tolerancia (`tolerance`) o hasta que el numero de iteraciones exceda el maximo (`maxiter`).

La funcion retorna un arreglo de todas las posiciones intermedias, y los valores de la funcion objetivo.

In [ ]:
def steepestdescent(objfun, gradient, init, tolerance=1e-6, maxiter=10000, steplength=0.01):
    p = init
    iterno=0
    parray = [p]
    fprev = objfun(p[0],p[1])
    farray = [fprev]
    while iterno < maxiter:
        p = p - steplength*gradient(p[0],p[1])
        fcur = objfun(p[0], p[1])
        if np.isnan(fcur):
            break
        parray.append(p)
        farray.append(fcur)
        if abs(fcur-fprev)<tolerance:
            break
        fprev = fcur
        iterno += 1
    return np.array(parray), np.array(farray)

Ahora veamos como el metodo steepest descent se comporta con la funcion Rosenbrock.

## Caso 1

In [ ]:
p, f = steepestdescent(objfun, gradient, init=[2,4], steplength=0.005)

Graficar la convergencia de la solucion. Izquierda: Los puntos solucion (blanco) superpuestas sobre el grafico de contorno. La estrella indica el punto inicial. Derecha: La funcion objetivo en cada iteración.

In [ ]:
plt.figure(figsize=(17,5))
plt.subplot(1,2,1)
contourplot(objfun, -1,3,0,10)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Minimize $f(x,y)=10(y-x^2)^2 + (1-x)^2$");
plt.scatter(p[0,0],p[0,1],marker="*",color="w")
for i in range(1,len(p)):    
        plt.plot( (p[i-1,0],p[i,0]), (p[i-1,1],p[i,1]) , "w");

plt.subplot(1,2,2)
plt.plot(f)
plt.xlabel("iteraciones")
plt.ylabel("$f(x,y)$");

La solucion es muy suave. Supongamos que se incrementa el tamaño del paso desde $\alpha=0.005$ a $\alpha=0.01$, y la trayectoria de la solucion se torna extraña.

In [ ]:
p, f = steepestdescent(objfun, gradient, init=[2,4], steplength=0.01)

In [ ]:
plt.figure(figsize=(17,5))
plt.subplot(1,2,1)
contourplot(objfun, -2,3,0,10)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Minimize $f(x,y)=10(y-x^2)^2 + (1-x)^2$");
plt.scatter(p[0,0],p[0,1],marker="*",color="w")
for i in range(1,len(p)):    
        plt.plot( (p[i-1,0],p[i,0]), (p[i-1,1],p[i,1]) , "w");

plt.subplot(1,2,2)
plt.plot(f)
plt.xlabel("iteraciones")
plt.ylabel("$f(x,y)$");

Ahora el tamaño del paso es mas largo, entonces la nueva posicion termina en una ubicacion donde el gradiente es mayor. Entonces, el siguiente paso es mas largo, y observamos un salto mas largo a traves de la montaña en la mitad.
Desde ahi el paso se vuelve mas pequeño otra vez, y la solucion se acerca al minimo global.

Intentar con valores diferentes de posicion inicial, donde el gradiente es mas grande. Ahora, el mismo $\alpha$ es muy grande; el tamaño del paso se incrementa en cada iteracion y el calculo explota!

In [ ]:
p, f = steepestdescent(objfun, gradient, init=[2,6], steplength=0.01)

Vemos que el valor de la funcion se incrementa rapidaente en cada iteración. El algoritmo es inestable.

In [ ]:
f

Sin embargo, cuando $\alpha$ se reduce a $0.005$ otra vez, vemos que la solucion converge edspues de algunas iteraciones.

In [ ]:
p, f = steepestdescent(objfun, gradient, init=[2,6], steplength=0.005)

In [ ]:
plt.figure(figsize=(17,5))
plt.subplot(1,2,1)
contourplot(objfun, -2,3,0,10)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Minimize $f(x,y)=10(y-x^2)^2 + (1-x)^2$");
plt.scatter(p[0,0],p[0,1],marker="*",color="w")
for i in range(1,len(p)):    
        plt.plot( (p[i-1,0],p[i,0]), (p[i-1,1],p[i,1]) , "w");

plt.subplot(1,2,2)
plt.plot(f)
plt.xlim(0,100)
plt.xlabel("iterations")
plt.ylabel("function value");

En general, la convergencia depende sensiblemente sobre el valor de $\alpha$, como tambien del valor local del gradiente en la posicion inicial. Puede jugar con diferentes posiciones iniciales y longitudes de paso para ver como funciona.

Tip: Por ensayo y error, intentar con valores de parametros que son muy cercanos a ser inestables; estos generan trayectorias locas. Por ejemplo:

In [ ]:
p, f = steepestdescent(objfun, gradient, init=[2,5.1155], steplength=0.01)

In [ ]:
plt.figure(figsize=(17,5))
plt.subplot(1,2,1)
contourplot(objfun, -3,3,0,10)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Minimize $f(x,y)=10(y-x^2)^2 + (1-x)^2$");
plt.scatter(p[0,0],p[0,1],marker="*",color="w")
for i in range(1,len(p)):    
        plt.plot( (p[i-1,0],p[i,0]), (p[i-1,1],p[i,1]) , "w");

plt.subplot(1,2,2)
plt.plot(f)
plt.xlabel("iteraciones")
plt.ylabel("$f(x,y)$");

## Ejercicios
1.  Se tiene la funcion objetivo
    $$f(x,y) = x^2 + 4y^2 + xy,$$
    cuyo gradiente es 
    $$\nabla f(x,y) = (2x + y,8y + x)^T.$$
    Esta es una función cuadrática con mínimo global en $(0,0)$, con valor $f(0,0)=0$. Encontrar el minimizador $\mathbf{x}^*$ de la función objetivo usando gradiente descendente.

2. Intentar ahora con la función $f(x,y) = |x| + |y|$, la cual no es suave. Ejecute el metodo de gradiente descendente; que observa? Optimizar tal funcion es tema de optimizacion "no-suave".
3. Intente minimizar la funcion no convexa con varios minimos y puntos de ensilladura. Por ejemplo, intentar con la funcion
$$f(x,y) = x^4 - 2x^2 + y^2.$$